In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import tensorflow as tf
from keras.src.layers import Embedding
from keras.src.legacy.preprocessing.text import Tokenizer
from keras.src.utils import pad_sequences
from nltk import word_tokenize
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dense, Lambda
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
import kagglehub
import nltk
from tensorflow.python.keras.utils.np_utils import to_categorical

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2025-06-22 16:44:32.972477: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750610673.308323      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750610673.400635      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/sentiment-analysis-dataset/training.1600000.processed.noemoticon.csv
/kaggle/input/sentiment-analysis-dataset/train.csv
/kaggle/input/sentiment-analysis-dataset/testdata.manual.2009.06.14.csv
/kaggle/input/sentiment-analysis-dataset/test.csv
/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin.gz
/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin
/kaggle/input/googlenewsvectors/GoogleNews-vectors-negative300.bin


In [2]:
nltk.download('punkt')
nltk.download('stopwords')


stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df_train = pd.read_csv('/kaggle/input/sentiment-analysis-dataset/train.csv', encoding='latin1')
df_test = pd.read_csv('/kaggle/input/sentiment-analysis-dataset/test.csv', encoding='latin1')

In [4]:
def remove_stopwords(text):
    tokens = word_tokenize(text.lower())  # tokenize and lowercase
    filtered = [word for word in tokens if word.isalnum() and word not in stop_words]
    return " ".join(filtered)

In [5]:
X_train = df_train[['text']]
y_train = df_train['sentiment'].map({'negative': 0, 'neutral': 1, 'positive': 2})  # or binary
df_test = df_test.dropna()
X_test = df_test[['text']]
y_test = df_test['sentiment'].map({'negative': 0, 'neutral': 1, 'positive': 2})  # Only if present

# Flatten the DataFrame columns
train_texts = X_train['text'].astype(str).apply(remove_stopwords)
test_texts = X_test['text'].astype(str).apply(remove_stopwords)

train_y = y_train.astype(int).tolist()
test_y = y_test.astype(int).tolist()

y_train_cat = to_categorical(train_y, num_classes=3)
y_test_cat = to_categorical(test_y, num_classes=3)

In [6]:
# Tokenize
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

max_len = max(max(len(seq) for seq in train_sequences), 50)

X_train_pad = pad_sequences(train_sequences, maxlen=max_len)
X_test_pad = pad_sequences(test_sequences, maxlen=max_len)

word_index = tokenizer.word_index

# Print first 5 keys
for key in list(word_index.keys())[:5]:
    print(key, word_index[key])

day 1
good 2
get 3
like 4
go 5


In [7]:
X_train_pad.shape

(27481, 50)

In [8]:
X_test_pad.shape

(3534, 50)

In [9]:
# Build Embedding Matrix from Word2Vec
path = kagglehub.dataset_download("leadbest/googlenewsvectorsnegative300")
path += '/GoogleNews-vectors-negative300.bin'
print("Path to dataset files:", path)
      
word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

embedding_dim = 300
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    if word in word2vec:
        embedding_matrix[i] = word2vec[word]

Path to dataset files: /kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin


In [10]:
embedding_matrix.shape

(23350, 300)

In [11]:
# Build LSTM Attention Sentiment Classifier
# Define the attention layer
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        self.u = None
        self.b = None
        self.W = None

    def build(self, input_shape):
        # Trainable weights for attention mechanism
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], input_shape[-1]),
                                 initializer="glorot_uniform", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(input_shape[-1],),
                                 initializer="zeros", trainable=True)
        self.u = self.add_weight(name="att_u", shape=(input_shape[-1],),
                                 initializer="glorot_uniform", trainable=True)

        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        # Score computation
        v = tf.tanh(tf.tensordot(inputs, self.W, axes=1) + self.b)
        vu = tf.tensordot(v, self.u, axes=1)
        alphas = tf.nn.softmax(vu)

        # Weighted sum of input
        output = tf.reduce_sum(inputs * tf.expand_dims(alphas, -1), axis=1)
        return output, alphas

# Sample Bi-LSTM model with Attention
def create_model(input_shape):
    inputs = Input(shape=input_shape)

    embedding_layer = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=max_len,
        trainable=False)(inputs)

    # Bi-LSTM layer
    lstm_out = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)

    # Add Attention layer
    attention_out, attention_weights = AttentionLayer()(lstm_out)

    reshaped = Lambda(lambda x: tf.expand_dims(x, axis=1))(attention_out)  # (batch, 1, features)

    # LSTM  layer post attention
    lstm_after_attn = LSTM(64, return_sequences=False)(reshaped)

    # flatten layer
    dense = Dense(128, activation='relu')(lstm_after_attn)

    # Final Dense layer
    outputs = Dense(3, activation='softmax')(dense)

    # Define the model
    return Model(inputs, outputs)

In [12]:
# Set input shape and compile the model
input_shape = (50,)  # For example, sequence length = 100, features per step = 50
model = create_model(input_shape)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-06-22 16:46:59.244878: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 50, 300)             │       7,005,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 50, 128)             │         186,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention_layer (AttentionLayer)     │ [(None, 128), (None, 50)]   │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda (Lambda)                      │ (None, 1, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,266,635 (27.72 MB)

 Trainable params: 261,635 (1022.01 KB)

 Non-trainable params: 7,005,000 (26.72 MB)

In [13]:
# Train Model
model.fit(X_train_pad, np.array(y_train_cat), epochs=50, batch_size=32, validation_split=0.2)

# Test Model
loss, accuracy = model.evaluate(X_test_pad, np.array(y_test_cat))
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 42s 51ms/step - accuracy: 0.4280 - loss: 1.0632 - val_accuracy: 0.5425 - val_loss: 0.9203
Epoch 2/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 35s 50ms/step - accuracy: 0.5862 - loss: 0.8918 - val_accuracy: 0.6296 - val_loss: 0.8466
Epoch 3/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 36s 53ms/step - accuracy: 0.6389 - loss: 0.8186 - val_accuracy: 0.6380 - val_loss: 0.8260
Epoch 4/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 34s 50ms/step - accuracy: 0.6522 - loss: 0.7968 - val_accuracy: 0.6454 - val_loss: 0.8201
Epoch 5/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 34s 50ms/step - accuracy: 0.6737 - loss: 0.7698 - val_accuracy: 0.6405 - val_loss: 0.8603
Epoch 6/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 34s 50ms/step - accuracy: 0.6780 - loss: 0.7523 - val_accuracy: 0.6478 - val_loss: 0.8160
Epoch 7/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 34s 50ms/step - accuracy: 0.6908 - loss: 0.7239 - val_accuracy: 0.6422 - val_loss: 0.8309
Epoch 8/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 34s 50ms/step - accuracy: 0.7057 - loss: 0.6980 - 